In [19]:
import os
import torch
import pandas as pd
import scanpy as sc
import time

In [22]:
import MVAADT

In [23]:
"""
Sets the device to GPU if available, otherwise defaults to CPU.
Also sets the environment variable 'R_HOME' to the specified path.

- `device`: A torch.device object set to 'cuda:1' if a GPU is available, otherwise 'cpu'.
- `os.environ['R_HOME']`: Sets the R_HOME environment variable to the specified path for R installation.
"""
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
os.environ['R_HOME'] = '/home/zxx/miniforge3/envs/MDI/lib/R'

In [24]:
# Specify data type
data_type = 'Spatial-epigenome-transcriptome'

# Fix random seed
from MVAADT.preprocess import fix_seed
random_seed = 2050
fix_seed(random_seed)

In [25]:
# read data
file_fold = '/home/zxx/MVAADT/data/Dataset5_Mouse_BrainP22/' #please replace 'file_fold' with the download path

adata_omics1 = sc.read_h5ad(file_fold + 'adata_RNA.h5ad')
adata_omics2 = sc.read_h5ad(file_fold + 'adata_peaks_normalized.h5ad')

adata_omics1.var_names_make_unique()
adata_omics2.var_names_make_unique()

In [26]:
from MVAADT.preprocess import clr_normalize_each_cell, pca, lsi

# RNA
sc.pp.filter_genes(adata_omics1, min_cells=10)
sc.pp.filter_cells(adata_omics1, min_genes=200)

sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata_omics1, target_sum=1e4)
sc.pp.log1p(adata_omics1)
sc.pp.scale(adata_omics1)

adata_omics1_high =  adata_omics1[:, adata_omics1.var['highly_variable']]
adata_omics1.obsm['feat'] = pca(adata_omics1_high, n_comps=50)

# ATAC
adata_omics2 = adata_omics2[adata_omics1.obs_names].copy() # .obsm['X_lsi'] represents the dimension reduced feature
if 'X_lsi' not in adata_omics2.obsm.keys():
    sc.pp.highly_variable_genes(adata_omics2, flavor="seurat_v3", n_top_genes=3000)
    lsi(adata_omics2, use_highly_variable=False, n_components=51)

adata_omics2.obsm['feat'] = adata_omics2.obsm['X_lsi'].copy()

In [27]:
print(adata_omics1.shape)
print(adata_omics2.shape)

(9196, 16304)
(9196, 121068)


In [9]:
from MVAADT.preprocess import mask_omics_data

# mask omics1 data
adata_omics1_masked, adata_omics1_remain, adata_omics1_miss = mask_omics_data(adata_omics1)

Number of samples: 9196
Number of mask samples: 1839
Masked indices: [3085 1877 2404 ... 6022 5590  706]
Masked indices number (1839,)
Spatial of masking: [[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Feat of masking: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Spatial after masking: [[ 0.  0.]
 [82. 91.]
 [81. 91.]
 ...
 [ 3. 92.]
 [ 0.  0.]
 [ 1. 92.]]
Feat after masking: [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 2.15943573  0.12342816  0.16797002 ...  0.06809265  0.3945665
  -0.08559207]
 [ 1.41322036 -0.51531836  0.43581047 ... -0.0847557   0.03057625
  -0.11642883]
 ...
 [-1.18687397 -1.19506717  1.64410296 ...  0.26342073  0.03525148
  -0.12239801]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 1.29295069 -0.02662093  0.20745309 ... -0.51382089  0.07339005
  -0.32658586]]
Remaining samples afte

In [10]:
from MVAADT.preprocess import construct_neighbor_graph
data = construct_neighbor_graph(adata_omics1_masked, adata_omics2, datatype=data_type)

In [ ]:
# define model
from MVAADT.MVAADT_partial import Train_MVAADT_partial
model = Train_MVAADT_partial(data, datatype=data_type, device=device)

start_time = time.time()

# train model
output = model.train()
end_time = time.time()
training_time = end_time - start_time

print(f"Training completed in {training_time} seconds.")

In [12]:
adata = adata_omics2.copy()
adata.obsm['GAN_Align'] = output['GAN_Align']
print(adata.obsm.keys())

KeysView(AxisArrays with keys: X_lsi, X_pca, X_umap, spatial, feat, GAN_Align)


In [13]:
from MVAADT.utils import clustering
tool = 'mclust' # mclust, leiden, and louvain
clustering(adata, key='GAN_Align', add_key='GAN_Align', n_clusters=18, method=tool, use_pca=False)

R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.1.1
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%


In [ ]:
# visualization
import matplotlib.pyplot as plt
fig, ax_list = plt.subplots(1, 2, figsize=(14, 5))
sc.pp.neighbors(adata, use_rep='GAN_Align', n_neighbors=30)
sc.tl.umap(adata)

sc.pl.umap(adata, color='GAN_Align', ax=ax_list[0], title='MVAADT(RNA Mask)', s=60, show=False)
sc.pl.embedding(adata, basis='spatial', color='GAN_Align', ax=ax_list[1], title='MVAADT(RNA Mask)', s=90, show=False)

plt.tight_layout(w_pad=0.3)
plt.show()

# 分别把左右两个图保存下来
fig.savefig('/home/zxx/MVAADT/result/Dataset5_Mouse_Brain_P22/MVAADT_RNA_masked.png',dpi=300)
fig.savefig('/home/zxx/MVAADT/result/Dataset5_Mouse_Brain_P22/MVAADT_RNA_masked.pdf',dpi=300)